In [90]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [91]:
from fastai.structured import *
from fastai.column_data import *
np.set_printoptions(threshold=50, edgeitems=20)

PATH='data/'

In [92]:
!ls {PATH}

googletrend.csv		   state_names.csv   test.csv	    weather.csv
rossmann.tgz		   store.csv	     test.csv.zip
sample_submission.csv	   store.csv.zip     train.csv
sample_submission.csv.zip  store_states.csv  train.csv.zip


# Create datasets

In [93]:
def concat_csv(dirname): #implementation used to get the above data set
    path = f'{PATH}{dirname}'
    filenames=glob(f"{PATH}/*.csv")
    
    wrote_header = False
    with open(f"{path}.csv","w") as outputfile: #??
            for filename in filenames:
                name = filename.split(".")[0]
                with open(filename) as f:
                    line = f.readline()
                    if not wrote_header:
                        wrote_header = True
                        outputfile.write("file,"+line)
                    for line in f:
                        outputfile.write(name + "," + line)
                    outputfile.write("\n")
        

In [94]:
# concat_csvs('googletrend')
# concat_csvs('weather')

In [95]:
table_names = ['train','store','store_states','state_names',
              'googletrend','weather','test']

Feature Space:

- train: Training set provided by competition
- store: List of stores
- store_states: mapping of store to the German state they are in
- List of German state names
- googletrend: trend of certain google keywords over time, found by users to correlate well w/ given data
- weather: weather
- test: testing set

In [96]:
tables = [pd.read_csv(f'{PATH}{fname}.csv',low_memory=False) for fname in table_names]

In [97]:
tables

[         Store  DayOfWeek        Date  Sales  Customers  Open  Promo  \
 0            1          5  2015-07-31   5263        555     1      1   
 1            2          5  2015-07-31   6064        625     1      1   
 2            3          5  2015-07-31   8314        821     1      1   
 3            4          5  2015-07-31  13995       1498     1      1   
 4            5          5  2015-07-31   4822        559     1      1   
 5            6          5  2015-07-31   5651        589     1      1   
 6            7          5  2015-07-31  15344       1414     1      1   
 7            8          5  2015-07-31   8492        833     1      1   
 8            9          5  2015-07-31   8565        687     1      1   
 9           10          5  2015-07-31   7185        681     1      1   
 10          11          5  2015-07-31  10457       1236     1      1   
 11          12          5  2015-07-31   8959        962     1      1   
 12          13          5  2015-07-31   8821      

In [9]:
from IPython.display import HTML

In [10]:
#use head() to display tables
for t in tables: display(t.head())

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


,Store,State
0,1,HE
1,2,TH
2,3,NW
3,4,BE
4,5,SN


,StateName,State
0,BadenWuerttemberg,BW
1,Bayern,BY
2,Berlin,BE
3,Brandenburg,BB
4,Bremen,HB


,file,week,trend
0,Rossmann_DE_SN,2012-12-02 - 2012-12-08,96
1,Rossmann_DE_SN,2012-12-09 - 2012-12-15,95
2,Rossmann_DE_SN,2012-12-16 - 2012-12-22,91
3,Rossmann_DE_SN,2012-12-23 - 2012-12-29,48
4,Rossmann_DE_SN,2012-12-30 - 2013-01-05,67


,file,Date,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Dew_PointC,MeanDew_PointC,Min_DewpointC,Max_Humidity,Mean_Humidity,...,Max_VisibilityKm,Mean_VisibilityKm,Min_VisibilitykM,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,Max_Gust_SpeedKm_h,Precipitationmm,CloudCover,Events,WindDirDegrees
0,NordrheinWestfalen,2013-01-01,8,4,2,7,5,1,94,87,...,31.0,12.0,4.0,39,26,58.0,5.08,6.0,Rain,215
1,NordrheinWestfalen,2013-01-02,7,4,1,5,3,2,93,85,...,31.0,14.0,10.0,24,16,NaN,0.00,6.0,Rain,225
2,NordrheinWestfalen,2013-01-03,11,8,6,10,8,4,100,93,...,31.0,8.0,2.0,26,21,NaN,1.02,7.0,Rain,240
3,NordrheinWestfalen,2013-01-04,9,9,8,9,9,8,100,94,...,11.0,5.0,2.0,23,14,NaN,0.25,7.0,Rain,263
4,NordrheinWestfalen,2013-01-05,8,8,7,8,7,6,100,94,...,10.0,6.0,3.0,16,10,NaN,0.00,7.0,Rain,268


,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0


In [11]:
#summarize aggregate information to each able accross each field
for t in tables: display(DataFrameSummary(t).summary())

AttributeError: module 'pandas.core.common' has no attribute 'is_numeric_dtype'

# Date Cleaning/Feature Engineering

In [98]:
train, store, store_states, state_names, googletrend, weather, test = tables

In [99]:
len(train),len(test)

(1017209, 41088)

In [100]:
#trun state Holiday to booleans
train.StateHoliday = train.StateHoliday!='0'
test.StateHoliday = test.StateHoliday!='0'

In [101]:
train.StateHoliday.head()

0    False
1    False
2    False
3    False
4    False
Name: StateHoliday, dtype: bool

In [102]:
#Pandas does joins using the merge method. The suffixes argument describes the naming convention for duplicate fields. 
#We've elected to leave the duplicate field names on the left untouched, and append a "_y" to those on the right.

def join_df(left, right, left_on, right_on=None, suffix='_y'):
    if right_on is None: right_on = left_on
    return left.merge(right, how='left', left_on = left_on, right_on=right_on,
                            suffixes=('',suffix))

In [103]:
weather = join_df(weather, state_names, 'file', 'StateName')

In [104]:
googletrend.head()

,file,week,trend
0,Rossmann_DE_SN,2012-12-02 - 2012-12-08,96
1,Rossmann_DE_SN,2012-12-09 - 2012-12-15,95
2,Rossmann_DE_SN,2012-12-16 - 2012-12-22,91
3,Rossmann_DE_SN,2012-12-23 - 2012-12-29,48
4,Rossmann_DE_SN,2012-12-30 - 2013-01-05,67


In [105]:
googletrend['Date'] = googletrend.week.str.split(' - ', expand=True)[0]
googletrend['State'] = googletrend.file.str.split('_', expand=True)[2]
#Use a boolean list to select all the NI values in the State column
#Using the .loc[rows, cols] feature to locate and replace selected values to 'HB,NI'
googletrend.loc[googletrend.State=='NI', 'State'] = 'HB,NI'

In [106]:
add_datepart(weather, 'Date', drop=False)
add_datepart(googletrend, 'Date', drop=False)
add_datepart(train, 'Date', drop=False)
add_datepart(test, 'Date', drop=False)

In [107]:
trend_de = googletrend[googletrend.file == 'Rossmann_DE']

In [108]:
store = join_df(store, store_states, "Store")
len(store[store.State.isnull()])

0

In [109]:
joined = join_df(train, store, "Store")
joined_test = join_df(test, store, "Store")
len(joined[joined.StoreType.isnull()]), len(joined_test[joined_test.StoreType.isnull()])

(0, 0)

In [40]:
googletrend.head()

,file,week,trend,Date,State,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,Rossmann_DE_SN,2012-12-02 - 2012-12-08,96,2012-12-02,SN,2012,12,48,2,6,337,False,False,False,False,False,False,1354406400
1,Rossmann_DE_SN,2012-12-09 - 2012-12-15,95,2012-12-09,SN,2012,12,49,9,6,344,False,False,False,False,False,False,1355011200
2,Rossmann_DE_SN,2012-12-16 - 2012-12-22,91,2012-12-16,SN,2012,12,50,16,6,351,False,False,False,False,False,False,1355616000
3,Rossmann_DE_SN,2012-12-23 - 2012-12-29,48,2012-12-23,SN,2012,12,51,23,6,358,False,False,False,False,False,False,1356220800
4,Rossmann_DE_SN,2012-12-30 - 2013-01-05,67,2012-12-30,SN,2012,12,52,30,6,365,False,False,False,False,False,False,1356825600


In [41]:
joined.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,...,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,State
0,1,5,2015-07-31,5263,555,1,1,False,1,2015,...,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,HE
1,2,5,2015-07-31,6064,625,1,1,False,1,2015,...,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",TH
2,3,5,2015-07-31,8314,821,1,1,False,1,2015,...,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct",NW
3,4,5,2015-07-31,13995,1498,1,1,False,1,2015,...,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN,BE
4,5,5,2015-07-31,4822,559,1,1,False,1,2015,...,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN,SN


In [110]:
joined = join_df(joined, googletrend, ['State', 'Year', 'Week'])
joined_test = join_df(joined_test, googletrend, ['State', 'Year', 'Week'])
len(joined[joined.trend.isnull()]), len(joined_test[joined_test.trend.isnull()])

(0, 0)

In [111]:
joined = joined.merge(trend_de, 'left', ['Year', 'Week'], suffixes=('','_DE'))
joined_test = joined_test.merge(trend_de, 'left', ['Year', 'Week'], suffixes=('','_DE'))
len(joined[joined.trend_DE.isnull()]), len(joined_test[joined_test.trend_DE.isnull()])

(0, 0)

In [112]:
joined = join_df(joined, weather, ['State','Date'])
joined_test = join_df(joined_test, weather, ["State","Date"])
len(joined[joined.Mean_TemperatureC.isnull()]), len(joined_test[joined_test.Mean_TemperatureC.isnull()])

(0, 0)

In [113]:
for df in (joined, joined_test):
    for c in df.columns:
        if c.endswith('_y'):
            if c in df.columns: df.drop(c, inplace=True, axis=1)

In [114]:
#dealing with missing values
for df in (joined, joined_test):
    df['CompetitionOpenSnceYear'] = df.CompetitionOpenSinceYear.fillna(1900).astype(np.int32)
    df['CompetitionOpenSinceMonth'] = df.CompetitionOpenSinceMonth.fillna(1).astype(np.int32)
    df['Promo2SinceYear'] = df.Promo2SinceYear.fillna(1900).astype(np.int32)
    df['Promo2SinceWeek'] = df.Promo2SinceWeek.fillna(1)

In [118]:
for df in (joined, joined_test):
    df['CompetitionOpenSince'] = pd.to_datetime(dict(year=df.CompetitionOpenSinceYear,
                                                    month=df.CompetitionOpenSinceMonth, day=15))
    df["CompetitionDaysOpen"] = df.Date.subtract(df.CompetitionOpenSince).dt.days

In [119]:
for df in (joined,joined_test): #?? Why assign value of zeros
    df.loc[df.CompetitionDaysOpen<0, "CompetitionDaysOpen"] = 0
    df.loc[df.CompetitionOpenSinceYear<1990, "CompetitionDaysOpen"] = 0

In [120]:
for df in (joined, joined_test):## issue here, inconsistent result from original notebook
    df["CompetitionMonthsOpen"] = df["CompetitionDaysOpen"]//30
    df.loc[df.CompetitionMonthsOpen>24, "CompetitionMonthsOpen"] = 24
joined.CompetitionMonthsOpen.unique()

array([24.,  3., 19.,  9., nan, 16., 17.,  7., 15., 22., 11., 13.,  2., 23.,  0., 12.,  4., 10.,  1., 14.,
       20.,  8., 18.,  6., 21.,  5.])